## Import librairies

In [1]:
import os
import sys

#Import config file. Update config.py according to your environment
import config

import pandas as pd
import numpy as np

import datetime

from Rakuten_preprocessing import Rakuten_img_path

from src.multimodal.classifiers import TFmultiClassifier

from src.utils.load import load_batch_results
from src.utils.batch import fit_save_all

/home/jul/anaconda3/envs/Rakuten/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

2024-03-20 10:39:23.974689: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-20 10:39:24.065838: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-20 10:39:24.065910: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-20 10:39:24.072983: E external/local_xla/xla/stream_execut

## Import pre-processed data

In [2]:
data_train = pd.read_csv(os.path.join(config.path_to_data, 'df_train_index.csv'))
data_train['testset'] = False
data_test = pd.read_csv(os.path.join(config.path_to_data, 'df_test_index.csv'))
data_test['testset'] = True
data = pd.concat([data_train, data_test], axis=0)

#merging text into token column
colnames = ['designation_translated', 'description_translated'] #['designation', 'description']#
data['tokens'] = data[colnames].apply(lambda row: ' '.join(s.lower() for s in row if isinstance(s, str)), axis=1)
    
#path to images into img_path column
data['img_path'] = Rakuten_img_path(img_folder=config.path_to_images,
                             imageid=data['imageid'], productid=data['productid'], suffix='_resized')


In [3]:
#labels of encoded classes
class_labels = data.groupby('prdtypedesignation')['prdtypeindex'].first().reset_index()
class_labels.index = class_labels['prdtypeindex']
class_labels = class_labels.drop(columns='prdtypeindex').sort_index()

## Create train and test sets

In [4]:
Img_train = data.loc[~data['testset'], 'img_path']
Img_test = data.loc[data['testset'], 'img_path']

Txt_train = data.loc[~data['testset'], 'tokens']
Txt_test = data.loc[data['testset'], 'tokens']

y_train = data.loc[~data['testset'],'prdtypeindex']
y_test = data.loc[data['testset'],'prdtypeindex']

#To be fed into any of our sklearn classifiers, X_train and X_test
#should be dataframes with columns tokens and img_path
X_train = pd.DataFrame({'tokens': Txt_train, 'img_path': Img_train})
X_test = pd.DataFrame({'tokens': Txt_test, 'img_path': Img_test})

#Number of classes
num_classes = len(np.unique(data['prdtypeindex']))

## Example usage: how to train TFmultiClassifier

In [ ]:
#defining callbacks
callbacks = []
callbacks.append(('EarlyStopping', {'monitor': 'val_accuracy', 'min_delta': 0, 'mode': 'max', 'patience': 2, 'restore_best_weights': True, 'verbose': 1}))

clf_multi = TFmultiClassifier(txt_base_name='camembert-base', img_base_name='vit_b16', 
                                 max_length=256, img_size=(224, 224, 3), augmentation_params=None,
                                 num_class=num_classes, drop_rate=0.2, attention_numheads=8, transfo_numblocks=3,
                                 epochs=8, batch_size=32, learning_rate=5e-5, validation_data=(X_test, y_test), callbacks=callbacks)

clf_multi.fit(X_train, y_train)
clf_multi.classification_score(X_test, y_test)
clf_multi.save('fusion/my_fusion_model')

## Transformer based fusion model

In [ ]:
#Name of the summary csv file to save results to
result_file_name = 'results_benchmark_fusion_TF.csv'

#type of modality
modality = 'fusion'

#Type of classifier
class_type = 'TFmultiClassifier'

#training parameters (or list of parameters for gridsearchCV)
num_class = num_classes
max_length = 256
n_epochs = 8
batch_size = 32
drop_rate = 0.2
lr0 = 5e-5
lr_min=5e-6
lr_decay_rate = 0.8

callbacks = []
#adding earlystopping callback
callbacks.append(('EarlyStopping', {'monitor': 'val_accuracy', 'min_delta': 0, 'mode': 'max', 'patience': 2, 'restore_best_weights': True, 'verbose': 1}))
#Adding tensorboard callback as the last one
callbacks.append(('TensorBoard', {'log_dir': np.nan, 'histogram_freq': 1, 'update_freq': 'epoch'}))

#grid search number of folds
nfolds_grid = 0

#cross-validation of f1-score
nfolds_cv = 0

#name of previously saved models to use as base estimators
base_name_list = ['camembert-base-ccnet vit_b16'] #['camembert-base vit_b16'] #['flaubert_base_uncased ResNet152']

#number of transformer blocks after fusion
transfo_numblocks = 6 #3 #0 #8
attn_numheads = 12 #8

#Initializing the list of parameters to batch over
params_list = []

for base_name in base_name_list:
  #Adjusting tensorboard log directory
  log_dir = os.path.join(config.path_to_tflogs, base_name, datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  callbacks[-1][1]['log_dir'] = log_dir
  #adding the set of parameters to the list
  [txt_base_name, img_base_name] = base_name.split()
  params_list.append({'modality': modality,
                      'class': class_type,
                      'base_name': base_name,
                      'model_suffix': 'TF' + str(transfo_numblocks) + '_att' + str(attn_numheads),
                      'param_grid': {'transfo_numblocks': transfo_numblocks, 'attention_numheads': attn_numheads, 'drop_rate': drop_rate, 
                                      'learning_rate': lr0, 'lr_decay_rate': lr_decay_rate, 'lr_min': lr_min,
                                      'max_length': max_length, 'num_class': num_class, 
                                      'epochs': n_epochs, 'batch_size': batch_size,
                                      'validation_data': (X_test, y_test), 'callbacks': [callbacks],
                                      'parallel_gpu': True},
                      'nfolds_grid': nfolds_grid, 'nfolds_cv': nfolds_cv
                      })
  
  
#Running the batch over params_list
results = fit_save_all(params_list, X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test, result_file_name = result_file_name)

## Load and check the saved result file

In [2]:
df_results = load_batch_results('results_benchmark_fusion_TF')
display(df_results)

,modality,class,vectorization,meta_method,classifier,tested_params,best_params,score_test,score_test_cat,conf_mat_test,score_train,fit_time,score_cv_test,score_cv_train,fit_cv_time,probs_test,pred_test,y_test,model_path
0,fusion,TFmultiClassifier,NaN,NaN,camembert-base vit_b16,"{'transfo_numblocks': [6], 'attention_numheads...",NaN,0.899074,"[0.7716417910447761, 0.8625235404896423, 0.985...","[[517, 0, 1, 2, 1, 3, 1, 0, 0, 1, 0, 0, 0, 1, ...",0.995640,31957.389296,NaN,NaN,NaN,"[[2.2252734197536483e-06, 2.4947016754595097e-...","[7, 10, 20, 2, 16, 0, 13, 20, 24, 23, 4, 15, 1...","[7, 10, 20, 2, 16, 0, 13, 20, 24, 23, 4, 15, 1...",fusion/camembert-base-vit_b16_TF6
1,fusion,TFmultiClassifier,NaN,NaN,camembert-base vit_b16,"{'transfo_numblocks': [3], 'attention_numheads...",NaN,0.897447,"[0.7662721893491125, 0.848893166506256, 0.9810...","[[518, 1, 2, 2, 2, 2, 2, 0, 1, 0, 0, 0, 0, 0, ...",0.997984,30475.240029,NaN,NaN,NaN,"[[1.0929949212368228e-07, 1.3302724255481735e-...","[7, 11, 20, 2, 16, 0, 13, 20, 24, 23, 4, 15, 1...","[7, 10, 20, 2, 16, 0, 13, 20, 24, 23, 4, 15, 1...",fusion/camembert-base-vit_b16_TF3_att12
2,fusion,TFmultiClassifier,NaN,NaN,camembert-base vit_b16,"{'transfo_numblocks': [6], 'attention_numheads...",NaN,0.898760,"[0.75625, 0.8589861751152075, 0.98356510745891...","[[484, 1, 1, 2, 3, 4, 1, 0, 0, 2, 0, 0, 0, 1, ...",0.989866,28874.559789,NaN,NaN,NaN,"[[6.817297980887815e-05, 3.961086463277752e-07...","[7, 10, 20, 2, 16, 0, 13, 20, 24, 23, 4, 15, 1...","[7, 10, 20, 2, 16, 0, 13, 20, 24, 23, 4, 15, 1...",fusion/camembert-base-vit_b16_TF6_att12
3,fusion,TFmultiClassifier,NaN,NaN,camembert-base vit_b16,"{'transfo_numblocks': [1], 'attention_numheads...",NaN,0.899024,"[0.7801418439716312, 0.8517110266159695, 0.977...","[[495, 1, 3, 4, 2, 6, 0, 1, 0, 0, 0, 1, 0, 2, ...",0.997704,20773.128222,NaN,NaN,NaN,"[[4.576388164423406e-06, 5.383282655202493e-07...","[7, 10, 20, 2, 16, 0, 13, 20, 24, 23, 4, 15, 1...","[7, 10, 20, 2, 16, 0, 13, 20, 24, 23, 4, 15, 1...",fusion/camembert-base-vit_b16_TF1_att12
